In [1]:
import polars as pl
import blitzbeaver as bb

In [4]:
csv_path = "/home/plouc314/Documents/epfl/ma1/shs/project/data/csv"

dataframes = [
    pl.read_csv(f"{csv_path}/1805.csv"),
    pl.read_csv(f"{csv_path}/1806.csv"),
    pl.read_csv(f"{csv_path}/1807.csv"),
]

In [2]:
# load the graph from a .beaver file
path_graph = "../graph.beaver"

graph = bb.TrackingGraph.load(path_graph)

In [3]:
record_schema = bb.RecordSchema(
    [
        bb.FieldSchema("chef_prenom_norm", bb.ElementType.String),
        bb.FieldSchema("chef_nom_norm", bb.ElementType.String),
    ]
)

In [4]:
config = bb.TrackingConfig(
    tracker=bb.TrackerConfig(
        tracker_type="simple",
        simple_tracker=bb.SimpleTrackerConfig(
            interest_threshold=0.7,
        ),
    ),
    distance_metric=bb.DistanceMetricConfig(
        metric="lvopti",
        caching_threshold=4,
    ),
    resolver=bb.ResolverConfig(
        resolving_strategy="simple",
    ),
)

In [5]:
# compute the graph
graph = bb.test_tracking_engine(config, record_schema, dataframes)
graph = bb.TrackingGraph(graph)

In [9]:
path_graph = "../graph.beaver"

graph.save(path_graph)

In [5]:
chain_id = graph._raw.root.outs[0][0]

graph.materialize_tracking_chain(chain_id, dataframes)

nom_rue,nom_rue_htr_corr,nom_rue_norm,no_maison,proprietaire_nom,proprietaire_nom_corr,proprietaire_nom_htr_corr,proprietaire_nom_norm,chef_prenom,chef_prenom_htr_corr,chef_prenom_norm,chef_nom,chef_nom_corr,chef_nom_htr_corr,chef_nom_norm,chef_annee_naissance,epouse_nom,epouse_nom_corr,epouse_nom_htr_corr,epouse_nom_norm,epouse_annee_naissance,fils_prenom,fils_annee_naissance,filles_prenom,filles_annee_naissance,chef_origine,chef_origine_corr,chef_origine_htr_corr,chef_origine_norm,chef_annee_arrivee,chef_vocation,chef_vocation_top_terms,chef_vocation_htr_corr,chef_vocation_norm,nb_domestiques,nb_ouvriers,nb_pensionnaires,observations,Page
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64
"""jn pierre""","""jn pierre""","""jean pierre""","""1""","""roguerol""","""roqueirol""","""roqueirol""","""roqueirol""","""damsan""","""damsan""","""damsan""","""roguerol""","""roqueirol""","""roqueirol""","""roqueirol""","""1756""","""·""","""·""","""·""","""·""","""·""","""·""","""·""","""·""","""·""","""des lausanne""","""des lausanne""","""des lausanne""","""des lausanne""","""nee""","""municipal""","""municipal""","""municipal""","""municipal""","""·""","""·""","""2""","""·""",1
"""st pierre""","""st pierre""","""st pierre""","""1""","""rogueirl""","""rogueirl""","""rogueirl""","""rogueirl""","""samson""","""samson""","""samson""","""roguerol""","""roqueirol""","""roqueirol""","""roqueirol""","""1756""","""·""","""·""","""·""","""·""","""·""","""·""","""·""","""·""","""·""","""de lausanne""","""de lausanne""","""de lausanne""","""de lausanne""","""ne""","""munieipal""","""munieipal""","""munieipal""","""munieipal""","""1""","""·""","""·""","""·""",1
"""st pierre""","""st pierre""","""st pierre""","""1""","""roguerrol""","""roguerrol""","""roguerrol""","""roguerrol""","""famson""","""famson""","""famson""","""roguerol""","""roqueirol""","""roqueirol""","""roqueirol""","""1736""","""·""","""·""","""·""","""·""","""·""","""·""","""·""","""·""","""·""","""de lausane""","""de lausane""","""de lausane""","""de lausane""","""ne""","""munieipal""","""munieipal""","""munieipal""","""munieipal""","""1""","""en""","""2""","""peis premr""",1


In [6]:
chain_metrics = bb.evaluate_tracking_chain_length(graph._raw)
print(chain_metrics)

graph_metrics = bb.evaluate_tracking_graph_properties(graph._raw)
print(graph_metrics)

EvalMetricChainLength(average=2.4358199, median=3, max=3, min=1)
EvalMetricGraphProperties(match_ratios=[0, 0.6256775, 0.49016282], conflict_ratios=[0, 0.10060976, 0.118385345])
